In [3]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import hist.dask as dhist
import dask
import pickle
import os
import dask_awkward as dak
import vector as sci_vector

In [4]:
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")

In [5]:
import coffea
print(coffea.__version__)
print(ak.__version__)
print(dak.__version__)

2024.1.1
2.5.2
2024.1.1


In [330]:
with open('../filelists/q347_files.txt', 'r') as f:
#with open('../filelists/hbb_files.txt', 'r') as f:
    q347_files = [line.strip() for line in f]
hbb = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/qcd/300to470/' + q347_files[1]: "/Events"},
    #delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "bkg"},
).events()

In [15]:
with open('../filelists/hgg_files.txt', 'r') as f:
    hgg_files = [line.strip() for line in f]
hgg = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/signal/hgg/' + hgg_files[0]: "/Events"},
    #delayed=False,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hgg"},
).events()

In [6]:
fatjet = hbb.FatJet
cut = ((fatjet.pt > 300) & (fatjet.msoftdrop > 110) & 
       (fatjet.msoftdrop < 140) & (abs(fatjet.eta) < 2.4)) #& (fatjet.btagDDBvLV2 > 0.89)
boosted_fatjet = hbb.FatJet[cut]
#slimmed = ak.mask(hbb.FatJet, cut)

In [7]:
def num_subjets(fatjet):
    pf = ak.flatten(fatjet.constituents.pf, axis=1)
    jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 0.4)
    cluster = fastjet.ClusterSequence(pf, jetdef)
    subjets = cluster.inclusive_jets()
    num = ak.num(subjets, axis=1)
    return num

In [8]:
num_sub = ak.unflatten(num_subjets(boosted_fatjet), counts=ak.num(boosted_fatjet))

In [9]:
boosted_fatjet['num_subjets'] = num_sub

In [10]:
ak.where(ak.flatten(boosted_fatjet.num_subjets.compute()) <= 2)

#--------------------------------------------------------------------------
#                         FastJet release 3.4.1
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


(<Array [21, 43, 53, 145, 159, ..., 1706, 1768, 1785, 1804] type='60 * int64'>,)

In [11]:
sub_cut = (boosted_fatjet.num_subjets >= 3)

In [12]:
a = num_sub.compute()

In [421]:
a = hgg.compute()

In [346]:
b = hbb.compute()

In [438]:
genelectron = b.GenPart[
    (abs(b.GenPart.pdgId) == 11)
    & b.GenPart.hasFlags(["isPrompt"])]

In [439]:
prompt = b.Electron.nearest(genelectron, threshold=0.2)

In [440]:
prompt_electron = ~ak.is_none(prompt, axis=1)

In [459]:
cut = (ak.fill_none(prompt.pt, 0, axis=1) < 20)

In [462]:
b.Electron[cut].pt

<Array [[199, 9.45], [15.8, ...], ..., []] type='25000 * var * float32[para...'>